In [ ]:
!tar xfvz /kaggle/input/ultralytics-for-offline-install/archive.tar.gz
!pip install --no-index --find-links=./packages ultralytics
!rm -rf ./packages

./packages/
./packages/networkx-3.4.2-py3-none-any.whl
./packages/fsspec-2025.2.0-py3-none-any.whl
./packages/python_dateutil-2.9.0.post0-py2.py3-none-any.whl
./packages/jinja2-3.1.5-py3-none-any.whl
./packages/pyparsing-3.2.1-py3-none-any.whl
./packages/charset_normalizer-3.4.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
./packages/ultralytics_thop-2.0.14-py3-none-any.whl
./packages/nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl
./packages/urllib3-2.3.0-py3-none-any.whl
./packages/nvidia_nvjitlink_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl
./packages/pytz-2025.1-py2.py3-none-any.whl
./packages/MarkupSafe-3.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
./packages/numpy-2.1.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


In [ ]:
!pip install --no-index --find-links=offline_packages onnxsim onnxruntime-gpu onnxslim


In [2]:
!cp -r /kaggle/input/junhans_yolo11m/other/default/1 onnx

In [3]:
!pip install ultralytics onnxslim onnxruntime-gpu onnxsim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.2/146.2 kB 3.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.2/283.2 MB 5.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 50.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.3 MB/s eta 0:00:00


In [4]:
from ultralytics import YOLO
model = YOLO("/kaggle/working/onnx/last.pt")
model.export(format="onnx", opset=12, device='cuda',dynamic=True,simplify=True)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.80 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
YOLO11m summary (fused): 125 layers, 20,030,803 parameters, 0 gradients, 67.6 GFLOPs

PyTorch: starting from '/kaggle/working/onnx/last.pt' with input shape (1, 3, 960, 960) BCHW and output shape(s) (1, 5, 18900) (38.7 MB)

ONNX: starting export with onnx 1.17.0 opset 12...
ONNX: slimming with onnxslim 0.1.56...
ONNX: export success ✅ 9.4s, saved as '/kaggle/working/onnx/last.onnx' (76.6 MB)

Export complete (12.5s)
Results saved to /kaggle/working/onnx
Predict:         yolo predict task=detect model=/kaggle/working/onnx/last.onnx imgsz=960  
Validate:        yolo val task=detect model=/kaggle/workin

'/kaggle/working/onnx/last.onnx'

In [5]:
model_path = "/kaggle/working/onnx/last.onnx"
CONFIDENCE_THRESHOLD = 0.3
MAX_DETECTIONS_PER_TOMO = 1
NMS_IOU_THRESHOLD = 0.2
CONCENTRATION = 0.5
BATCH_SIZE = 8 

import os
from pathlib import Path
import os
import numpy as np
import pandas as pd
from PIL import Image
import torch
import cv2
from tqdm.notebook import tqdm
from ultralytics import YOLO
import threading
import time
from contextlib import nullcontext
from concurrent.futures import ThreadPoolExecutor

from pathlib import Path

from torch.utils.data import Dataset, DataLoader
from torch.utils.data import DataLoader, TensorDataset



np.random.seed(42)
torch.manual_seed(42)


data_path = "/kaggle/input/byu-locating-bacterial-flagellar-motors-2025/"
test_dir = os.path.join(data_path, "test")
submission_path = "/kaggle/working/submission.csv"

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
if device.startswith('cuda'):
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = False
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

# AMP scaler for inference (no scaling)
scaler = torch.cuda.amp.GradScaler(enabled=False)

class GPUProfiler:
    def __init__(self, name):
        self.name = name
        self.start_time = None
    def __enter__(self):
        if torch.cuda.is_available(): torch.cuda.synchronize()
        self.start_time = time.time()
    def __exit__(self, *args):
        if torch.cuda.is_available(): torch.cuda.synchronize()
        elapsed = time.time() - self.start_time
        # print(f"[PROFILE] {self.name}: {elapsed:.3f}s")


def perform_3d_nms(detections, iou_threshold):
    """
    Perform 3D Non-Maximum Suppression on detections to merge nearby motors
    """
    if not detections:
        return []
    
    # Sort by confidence (highest first)
    detections = sorted(detections, key=lambda x: x['confidence'], reverse=True)
    
    # List to store final detections after NMS
    final_detections = []
    
    # Define 3D distance function
    def distance_3d(d1, d2):
        return np.sqrt((d1['z'] - d2['z'])**2 + 
                       (d1['y'] - d2['y'])**2 + 
                       (d1['x'] - d2['x'])**2)
    
    # Maximum distance threshold (based on box size and slice gap)
    box_size = 24  # Same as annotation box size
    distance_threshold = box_size * iou_threshold
    
    # Process each detection
    while detections:
        # Take the detection with highest confidence
        best_detection = detections.pop(0)
        final_detections.append(best_detection)
        return final_detections
    #     # Filter out detections that are too close to the best detection
        # detections = [d for d in detections if distance_3d(d, best_detection) > distance_threshold]
    
    return final_detections


from concurrent.futures import ThreadPoolExecutor

class TomogramSliceDataset(Dataset):
    def __init__(self, paths, batch_size):
        self.paths = paths
        self.batch_size = batch_size

    def __len__(self):
        return len(self.paths) // self.batch_size

    def load_and_process(self, p):
        img = cv2.imread(p)
        # img = cv2.medianBlur(img, ksize=5)
        return img
    def __getitem__(self, idx):
        start = idx * self.batch_size
        end = (idx + 1) * self.batch_size
        p_batch = self.paths[start:end]
        with ThreadPoolExecutor(max_workers=14) as executor:
            imgs = list(executor.map(self.load_and_process, p_batch))
    
        return list(range(start, end)), imgs

    @staticmethod
    def collate_fn(batch):
        paths, imgs = zip(*batch)
        return list(paths), list(imgs)




@torch.no_grad()
def process_tomogram(tomo_id, model, total, idx):
    
    tomo_path = os.path.join(test_dir, tomo_id)
    files = sorted([f for f in os.listdir(tomo_path) if f.endswith('.jpg')])
    sel = np.linspace(0, len(files)-1, int(len(files)*CONCENTRATION)).round().astype(int)
    files = [files[i] for i in sel]
    paths = [os.path.join(tomo_path, f) for f in files]
    slice_nums = [int(f.split('_')[1].split('.')[0]) for f in files]

    ds = TomogramSliceDataset(paths, batch_size=32)
    loader = DataLoader(ds, batch_size=1, shuffle=False,
                        num_workers=1, pin_memory=True,
                        collate_fn=TomogramSliceDataset.collate_fn)
    

    all_dets = []
    # volm = []
    for batch_zidx, imgs in loader:
        imgs = imgs[0]
        batch_zidx = batch_zidx[0]
        # print(len(imgs))
        with GPUProfiler(f"Inference {len(batch_zidx)} slices"):
            with torch.cuda.amp.autocast():
                # print(imgs[0].shape)
                    # model.predict(source="img.jpg", device=0)

                preds = model.predict(imgs,device='cuda', verbose=False)
        torch.cuda.empty_cache()
        for z, result in zip(batch_zidx, preds):
            z = slice_nums.pop(0)
            if len(result.boxes) > 0 and result.boxes.conf[0] >= CONFIDENCE_THRESHOLD :
                x1,y1,x2,y2 = result.boxes.xyxy[0].cpu().numpy()
                all_dets.append({
                            'z': z,
                            'y': int(round((y1+y2)/2)),
                            'x': int(round((x1+x2)/2)),
                            'confidence': float(result.boxes.conf[0])
                        })
           
    # if device.startswith('cuda'): torch.cuda.synchronize()
    final = perform_3d_nms(all_dets, NMS_IOU_THRESHOLD)
    # final.sort(key=lambda x: x['confidence'], reverse=True)
    if not final:
        return {'tomo_id': tomo_id, 'Motor axis 0': -1,
                'Motor axis 1': -1, 'Motor axis 2': -1}
        
    # if len(all_dets)<2:
    #     return {'tomo_id': tomo_id, 'Motor axis 0': -1,
    #             'Motor axis 1': -1, 'Motor axis 2': -1}
        
    best = final[0]
    return {'tomo_id': tomo_id,
            'Motor axis 0': best['z'],
            'Motor axis 1': best['y'],
            'Motor axis 2': best['x']}


def generate_submission():
    tomos = sorted([d for d in os.listdir(test_dir) if os.path.isdir(os.path.join(test_dir, d))])
    model = YOLO(model_path)
    # model.to(device)
    # if device.startswith('cuda'):
    #     model.fuse()
    #     if torch.cuda.get_device_capability(0)[0] >= 7:
    #         model.model.half()

    results = []
    for idx, tomo in enumerate(tqdm(tomos, desc="Tomo loop"), 1):
        torch.cuda.empty_cache()
        res = process_tomogram(tomo, model, len(tomos), idx)
        results.append(res)

    df = pd.DataFrame(results)[['tomo_id','Motor axis 0','Motor axis 1','Motor axis 2']]
    df.to_csv(submission_path, index=False)
    print(df.head())
    return df

if __name__ == "__main__":
    start = time.time()
    generate_submission()
    print(f"Total time: {time.time()-start:.2f}s")


WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.


<ipython-input-5-650aa9cef167>:46: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=False)


Tomo loop:   0%|          | 0/3 [00:00<?, ?it/s]

Ultralytics 8.3.80 🚀 Python-3.10.12 torch-2.5.1+cu121 


<ipython-input-5-650aa9cef167>:147: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


ValueError: Invalid CUDA 'device=0,1' requested. Use 'device=cpu' or pass valid CUDA device(s) if available, i.e. 'device=0' or 'device=0,1,2,3' for Multi-GPU.

torch.cuda.is_available(): True
torch.cuda.device_count(): 1
os.environ['CUDA_VISIBLE_DEVICES']: 0


In [ ]:
model_path = "/kaggle/input/junhans_yolo11m/other/default/1/last.pt"
CONFIDENCE_THRESHOLD = 0.7
MAX_DETECTIONS_PER_TOMO = 1
NMS_IOU_THRESHOLD = 0.2
CONCENTRATION = 0.5
BATCH_SIZE = 8 

import os
from pathlib import Path

# current_dir = Path.cwd()
# print("this_dir:", current_dir)

# target_dir = Path("/kaggle/working/MHAF-YOLO-main") 
# os.chdir(target_dir)  

import os
import numpy as np
import pandas as pd
from PIL import Image
import torch
import cv2
from tqdm.notebook import tqdm
from ultralytics import YOLO
import threading
import time
from contextlib import nullcontext
from concurrent.futures import ThreadPoolExecutor

from pathlib import Path

from torch.utils.data import Dataset, DataLoader
from torch.utils.data import DataLoader, TensorDataset



np.random.seed(42)
torch.manual_seed(42)


data_path = "/kaggle/input/byu-locating-bacterial-flagellar-motors-2025/"
test_dir = os.path.join(data_path, "test")
submission_path = "/kaggle/working/submission.csv"

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
if device.startswith('cuda'):
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = False
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

# AMP scaler for inference (no scaling)
scaler = torch.cuda.amp.GradScaler(enabled=False)

class GPUProfiler:
    def __init__(self, name):
        self.name = name
        self.start_time = None
    def __enter__(self):
        if torch.cuda.is_available(): torch.cuda.synchronize()
        self.start_time = time.time()
    def __exit__(self, *args):
        if torch.cuda.is_available(): torch.cuda.synchronize()
        elapsed = time.time() - self.start_time
        # print(f"[PROFILE] {self.name}: {elapsed:.3f}s")


def perform_3d_nms(detections, iou_threshold):
    """
    Perform 3D Non-Maximum Suppression on detections to merge nearby motors
    """
    if not detections:
        return []
    
    # Sort by confidence (highest first)
    detections = sorted(detections, key=lambda x: x['confidence'], reverse=True)
    
    # List to store final detections after NMS
    final_detections = []
    
    # Define 3D distance function
    def distance_3d(d1, d2):
        return np.sqrt((d1['z'] - d2['z'])**2 + 
                       (d1['y'] - d2['y'])**2 + 
                       (d1['x'] - d2['x'])**2)
    
    # Maximum distance threshold (based on box size and slice gap)
    box_size = 24  # Same as annotation box size
    distance_threshold = box_size * iou_threshold
    
    # Process each detection
    while detections:
        # Take the detection with highest confidence
        best_detection = detections.pop(0)
        final_detections.append(best_detection)
        return final_detections
    #     # Filter out detections that are too close to the best detection
        # detections = [d for d in detections if distance_3d(d, best_detection) > distance_threshold]
    
    return final_detections


class TomogramSliceDataset(Dataset):
    def __init__(self, paths, target_size=(960,960)):
        self.paths = paths
        self.target_size = target_size
    def __len__(self):
        return len(self.paths)
    def __getitem__(self, idx):   
        p = self.paths[idx]
        img = cv2.imread(p)
        img = cv2.medianBlur(img, ksize=5)  # Kernel size must be odd
        if img is None:
            img = np.array(Image.open(p))
        # Resize to target size to limit GPU memory usage
        # img = cv2.resize(img, self.target_size, interpolation=cv2.INTER_AREA)
        # img = torch.from_numpy(img)
        return idx, img

    @staticmethod
    def collate_fn(batch):
        paths, imgs = zip(*batch)
        return list(paths), list(imgs)



@torch.no_grad()
def process_tomogram(tomo_id, model, total, idx):
    tomo_path = os.path.join(test_dir, tomo_id)
    files = sorted([f for f in os.listdir(tomo_path) if f.endswith('.jpg')])
    sel = np.linspace(0, len(files)-1, int(len(files)*CONCENTRATION)).round().astype(int)
    files = [files[i] for i in sel]
    paths = [os.path.join(tomo_path, f) for f in files]
    slice_nums = [int(f.split('_')[1].split('.')[0]) for f in files]

    ds = TomogramSliceDataset(paths)
    loader = DataLoader(ds, batch_size=32, shuffle=False,
                        num_workers=4, pin_memory=True,
                        collate_fn=TomogramSliceDataset.collate_fn)

    all_dets = []
    # volm = []
    for batch_zidx, imgs in loader:
        with GPUProfiler(f"Inference {len(batch_zidx)} slices"):
            with torch.cuda.amp.autocast():
                preds = model(imgs, verbose=False)
        
        # imgs = np.concatenate(imgs, axis=0)
        # (B,H, W) = im/gs.shape
        # volm.append(imgs.reshape(-1, H, W))
        for z, result in zip(batch_zidx, preds):
            z = slice_nums.pop(0)
            if len(result.boxes) > 0 and result.boxes.conf[0] >= CONFIDENCE_THRESHOLD :
                x1,y1,x2,y2 = result.boxes.xyxy[0].cpu().numpy()
                all_dets.append({
                            'z': z,
                            'y': int(round((y1+y2)/2)),
                            'x': int(round((x1+x2)/2)),
                            'confidence': float(result.boxes.conf[0])
                        })
           
    # if device.startswith('cuda'): torch.cuda.synchronize()
    final = perform_3d_nms(all_dets, NMS_IOU_THRESHOLD)
    # final.sort(key=lambda x: x['confidence'], reverse=True)
    if not final:
        return {'tomo_id': tomo_id, 'Motor axis 0': -1,
                'Motor axis 1': -1, 'Motor axis 2': -1}
        
    # if len(all_dets)<2:
    #     return {'tomo_id': tomo_id, 'Motor axis 0': -1,
    #             'Motor axis 1': -1, 'Motor axis 2': -1}
        
    best = final[0]
    return {'tomo_id': tomo_id,
            'Motor axis 0': best['z'],
            'Motor axis 1': best['y'],
            'Motor axis 2': best['x']}


def generate_submission():
    tomos = sorted([d for d in os.listdir(test_dir) if os.path.isdir(os.path.join(test_dir, d))])
    model = YOLO(model_path)
    model.to(device)
    if device.startswith('cuda'):
        model.fuse()
        if torch.cuda.get_device_capability(0)[0] >= 7:
            model.model.half()

    results = []
    for idx, tomo in enumerate(tqdm(tomos, desc="Tomo loop"), 1):
        torch.cuda.empty_cache()
        res = process_tomogram(tomo, model, len(tomos), idx)
        results.append(res)

    df = pd.DataFrame(results)[['tomo_id','Motor axis 0','Motor axis 1','Motor axis 2']]
    df.to_csv(submission_path, index=False)
    print(df.head())
    return df

if __name__ == "__main__":
    start = time.time()
    generate_submission()
    print(f"Total time: {time.time()-start:.2f}s")


In [ ]:
# !cp -r /kaggle/input/mhafyolo/pytorch/default/1/MHAF-YOLO-main /kaggle/working/
# model_path = "/kaggle/input/mhaf_2.5d/onnx/default/1/best (11).pt"
# CONFIDENCE_THRESHOLD = 0.3
# MAX_DETECTIONS_PER_TOMO = 1 # This seems to be handled by perform_3d_nms returning only one
# NMS_IOU_THRESHOLD = 0.2
# CONCENTRATION = 1 # This determines how many center slices we'll consider
# BATCH_SIZE = 8 # Adjust based on GPU memory for 3-channel images
# STACK_STRIDE = 2 # For z-2, z, z+2. Results in a 3-channel image.
# # If you change STACK_STRIDE, ensure your model's first layer can handle the number of input channels if it's not 3.
# # For this example, we are creating a 3-channel image.

# import os
# from pathlib import Path
# import shutil # For potential future use

# current_dir = Path.cwd()
# print("this_dir:", current_dir)

# target_dir = Path("/kaggle/working/MHAF-YOLO-main")
# # if not target_dir.exists():
# #     # Assuming you have a way to get MHAF-YOLO-main, e.g., from another input or git clone
# #     # For now, let's mock its creation if it's just about running the script
# #     # In a real Kaggle notebook, you'd copy it from an input source
# #     print(f"{target_dir} does not exist. Please ensure it's available or copied.")
# #     # Example: shutil.copytree("/kaggle/input/your-mhaf-yolo-repo", target_dir)
# #     target_dir.mkdir(parents=True, exist_ok=True) # Create if not exists for now
# # else:
# #     print(f"Target directory {target_dir} already exists.")

# os.chdir(target_dir)
# print(f"Changed CWD to: {Path.cwd()}")


# import numpy as np
# import pandas as pd
# from PIL import Image
# import torch
# import cv2
# from tqdm.notebook import tqdm
# from ultralytics import YOLOv10 # Assuming this is the correct import for your model
# import threading # Not actively used in the final version, can be removed
# import time
# from contextlib import nullcontext # Not actively used, can be removed
# from concurrent.futures import ThreadPoolExecutor # Not actively used, can be removed

# from torch.utils.data import Dataset, DataLoader
# # from torch.utils.data import DataLoader, TensorDataset # Redundant import

# np.random.seed(42)
# torch.manual_seed(42)

# data_path = "/kaggle/input/byu-locating-bacterial-flagellar-motors-2025/"
# test_dir = os.path.join(data_path, "test")
# submission_path = "/kaggle/working/submission.csv"

# device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
# if device.startswith('cuda'):
#     torch.backends.cudnn.benchmark = True
#     torch.backends.cudnn.deterministic = False # Usually False for speed
#     torch.backends.cuda.matmul.allow_tf32 = True
#     torch.backends.cudnn.allow_tf32 = True

# # AMP scaler for inference (no scaling if enabled=False)
# scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available()) # Enable if using AMP

# class GPUProfiler:
#     def __init__(self, name):
#         self.name = name
#         self.start_time = None
#     def __enter__(self):
#         if torch.cuda.is_available(): torch.cuda.synchronize()
#         self.start_time = time.time()
#     def __exit__(self, *args):
#         if torch.cuda.is_available(): torch.cuda.synchronize()
#         elapsed = time.time() - self.start_time
#         # print(f"[PROFILE] {self.name}: {elapsed:.3f}s")


# def perform_3d_nms(detections, iou_threshold):
#     if not detections:
#         return []
#     detections = sorted(detections, key=lambda x: x['confidence'], reverse=True)
#     final_detections = []
    
#     # Simplified NMS: if we only want MAX_DETECTIONS_PER_TOMO = 1,
#     # we just take the highest confidence one after initial filtering.
#     # The original code for NMS was more complex than needed if only one detection is kept.
#     if detections: # If there's at least one detection
#         final_detections.append(detections[0]) # Take the best one
#         return final_detections


# class TomogramSliceDataset25D(Dataset):
#     def __init__(self, tomo_id_path, center_slice_filenames_with_paths, all_slices_in_tomo_map, 
#                  stack_stride, target_size=(960, 960)):
#         self.tomo_id_path = tomo_id_path # Path to the tomogram directory
#         self.center_slice_info = center_slice_filenames_with_paths # List of (abs_path, slice_idx) for center slices
#         self.all_slices_in_tomo_map = all_slices_in_tomo_map # Map: slice_idx -> abs_path for all slices
#         self.stack_stride = stack_stride
#         self.target_size = target_size
#         self.num_channels = 3 # Since we are stacking 3 slices (z-stride, z, z+stride)

#     def __len__(self):
#         return len(self.center_slice_info)

#     def _load_slice(self, slice_idx):
#         """Loads a single slice, handling boundaries by clamping and reusing edge slices."""
#         # Find the closest available slice if exact index is out of bounds for stacking
#         min_available_idx = min(self.all_slices_in_tomo_map.keys())
#         max_available_idx = max(self.all_slices_in_tomo_map.keys())
        
#         clamped_idx = np.clip(slice_idx, min_available_idx, max_available_idx)
        
#         slice_path = self.all_slices_in_tomo_map.get(clamped_idx)

#         if slice_path is None: # Should not happen if map is complete for the tomogram
#             print(f"Warning: Slice for index {clamped_idx} (original {slice_idx}) not found in map. Returning zeros.")
#             return np.zeros((*self.target_size, 1), dtype=np.uint8) # H, W, C (grayscale)

#         try:
#             # img = cv2.imread(slice_path, cv2.IMREAD_GRAYSCALE) # Read as grayscale
#             # Using PIL as it was in original snippet for one path
#             img = Image.open(slice_path) # Convert to grayscale
#             img = np.array(img)

#             if img is None:
#                 raise ValueError(f"cv2.imread failed for {slice_path}")
#             # return img
#             # if img.shape[:2] != self.target_size: # Check if resize is needed
#                  # img = cv2.resize(img, self.target_size, interpolation=cv2.INTER_AREA)
#             return img[:, :, np.newaxis] # Add channel dim: H, W, 1
#         except Exception as e:
#             print(f"Error loading/processing slice {slice_path} for index {slice_idx}: {e}. Returning zeros.")
#             return np.zeros((*self.target_size, 1), dtype=np.uint8)


#     def __getitem__(self, idx):
#         center_slice_path, center_slice_z_idx = self.center_slice_info[idx]
        
#         slices_to_stack = []
#         # Slice indices for the stack: [z - stride, z, z + stride]
#         slice_indices = [
#             center_slice_z_idx - self.stack_stride,
#             center_slice_z_idx,
#             center_slice_z_idx + self.stack_stride,
#         ]

#         for z_val in slice_indices:
#             slices_to_stack.append(self._load_slice(z_val))
        
#         # Stack along the channel dimension (last dimension)
#         stacked_img = np.concatenate(slices_to_stack, axis=2) # Shape: H, W, 3

#         # YOLO typically expects CHW format if using PyTorch transforms,
#         # but the model.predict() often handles HWC for NumPy arrays.
#         # If you need to convert to CHW:
#         # stacked_img = np.transpose(stacked_img, (2, 0, 1)) # Shape: 3, H, W
#         # stacked_img = torch.from_numpy(stacked_img).float() / 255.0 # Normalize if needed

#         # The model.predict() in ultralytics often takes HWC numpy arrays.
#         return center_slice_z_idx, stacked_img # Return center_z_idx for this stack

#     @staticmethod
#     def collate_fn(batch):
#         # center_z_indices, stacked_images = zip(*batch)
#         # return list(center_z_indices), list(stacked_images) # Pass as list of numpy arrays
#         # Default collate should work if __getitem__ returns (int, np.ndarray)
#         # It will stack the numpy arrays along a new batch dimension.
#         # If returning PyTorch tensors, default collate is also fine.
#         center_z_indices, stacked_images_list = zip(*batch)
#         # Assuming stacked_images_list contains HWC numpy arrays
#         # Convert to a single batch tensor if model expects that, or pass list
#         return list(center_z_indices), stacked_images_list


# @torch.no_grad()
# def process_tomogram(tomo_id, model, total_tomos, current_tomo_idx): # Renamed args for clarity
#     tomo_dir_path = Path(test_dir) / tomo_id
    
#     # Get all available .jpg slices and their integer indices for this tomogram
#     all_slice_files = sorted(tomo_dir_path.glob('*.jpg'))
#     if not all_slice_files:
#         print(f"No jpg files found in {tomo_dir_path}")
#         return {'tomo_id': tomo_id, 'Motor axis 0': -1, 'Motor axis 1': -1, 'Motor axis 2': -1}

#     all_slices_map = {} # Map from Z-index to absolute path
#     for f_path in all_slice_files:
#         try:
#             slice_idx = int(f_path.stem.split('_')[1])
#             all_slices_map[slice_idx] = str(f_path)
#         except (IndexError, ValueError):
#             print(f"Could not parse slice index from filename: {f_path.name}")
#             continue
    
#     if not all_slices_map:
#         print(f"Could not parse any slice indices for tomogram {tomo_id}")
#         return {'tomo_id': tomo_id, 'Motor axis 0': -1, 'Motor axis 1': -1, 'Motor axis 2': -1}

#     # Determine which slices will be the *center* of a 2.5D stack
#     # These are the Z-coordinates we will effectively "scan"
#     # CONCENTRATION determines sampling of these center slices
#     center_slice_indices_available = sorted(all_slices_map.keys())
#     num_center_slices_to_consider = int(len(center_slice_indices_available) * CONCENTRATION)
#     if num_center_slices_to_consider == 0 and len(center_slice_indices_available) > 0:
#         num_center_slices_to_consider = 1 # Ensure at least one if possible

#     selected_center_indices_for_stacking = np.linspace(
#         0, 
#         len(center_slice_indices_available) - 1, 
#         num_center_slices_to_consider
#     ).round().astype(int)
    
#     center_slices_info_for_dataset = [] # List of (abs_path_of_center_slice, center_slice_z_idx)
#     for sel_idx in selected_center_indices_for_stacking:
#         actual_z_idx = center_slice_indices_available[sel_idx]
#         center_slices_info_for_dataset.append((all_slices_map[actual_z_idx], actual_z_idx))

#     if not center_slices_info_for_dataset:
#         print(f"No center slices selected for processing for tomogram {tomo_id}")
#         return {'tomo_id': tomo_id, 'Motor axis 0': -1, 'Motor axis 1': -1, 'Motor axis 2': -1}

#     # Create dataset and dataloader for 2.5D stacks
#     # Hardcoding target_size, can be a parameter
#     target_img_size = (model.model.yaml.get('imgsz', [960, 960]))[0] # Get imgsz from model if possible
#     if isinstance(target_img_size, list): target_img_size = target_img_size[0] # e.g. [960,960]
#     # If your images are not square, and model expects square, you might need padding.
#     # For now, assuming resize to a square shape like (960,960) or (960,960) from original TomogramSliceDataset.
#     # Using (960,960) from original TomogramSliceDataset. For YOLO, it's often (960,960)
#     # Let's use a common YOLO size, but ensure your model is trained/fine-tuned on it.
#     # For inference, it's best to match training size.
#     # The model.predict() usually handles resizing internally if you provide `imgsz` argument.
#     # Here, we are pre-resizing in the dataset. This might be useful if original images are huge.
#     # Let's make target_size consistent with a typical YOLO input.
#     # For consistency with original code, use (960,960), but be mindful of model training size.
#     # For YOLO, often 960. If model.predict resizes, then dataset resize is redundant.
#     # For now, let's assume we do want to resize in dataset.
#     dataset_target_size = (960, 960) # Example, adjust as needed for your model and performance.

#     ds = TomogramSliceDataset25D(str(tomo_dir_path), center_slices_info_for_dataset, all_slices_map, 
#                                  STACK_STRIDE, target_size=dataset_target_size)
    
#     loader = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=False,
#                         num_workers=min(os.cpu_count(), 4), # More robust num_workers
#                         pin_memory=True if device.startswith('cuda') else False,
#                         collate_fn=TomogramSliceDataset25D.collate_fn)

#     all_dets = []
#     for batch_center_z_indices, batch_stacked_imgs_list in loader:
#         # batch_stacked_imgs_list is a list of HWC numpy arrays
#         # model.predict can usually take a list of images
#         with GPUProfiler(f"Inference {len(batch_stacked_imgs_list)} stacks"):
#             # AMP context manager for inference
#             with torch.cuda.amp.autocast(enabled=device.startswith('cuda')):
#                  # verbose=False, imgsz=dataset_target_size (or let model use its default)
#                 preds = model(batch_stacked_imgs_list, verbose=False)
        
#         for center_z, result in zip(batch_center_z_indices, preds):
#             # 'center_z' is the Z-index of the central slice of the stack
#             if len(result.boxes) > 0 and result.boxes.conf[0] >= CONFIDENCE_THRESHOLD:
#                 x1,y1,x2,y2 = result.boxes.xyxy[0].cpu().numpy()
#                 final_y = int(round((y1+y2)/2))
#                 final_x = int(round((x1+x2)/2))

#                 all_dets.append({
#                     'z': center_z, # Z-coordinate of the center of the stack
#                     'y': final_y,
#                     'x': final_x,
#                     'confidence': float(result.boxes.conf[0])
#                 })
           
#     final_detections_after_nms = perform_3d_nms(all_dets, NMS_IOU_THRESHOLD)
    
#     if not final_detections_after_nms:
#         return {'tomo_id': tomo_id, 'Motor axis 0': -1,
#                 'Motor axis 1': -1, 'Motor axis 2': -1}
        
#     best_detection = final_detections_after_nms[0]
#     return {'tomo_id': tomo_id,
#             'Motor axis 0': best_detection['z'],
#             'Motor axis 1': best_detection['y'],
#             'Motor axis 2': best_detection['x']}


# def generate_submission():
#     tomos = sorted([d for d in os.listdir(test_dir) if os.path.isdir(os.path.join(test_dir, d))])
#     # tomos = tomos[:3] # For quick testing

#     print(f"Loading model from {model_path}...")
#     model = YOLOv10(model_path) # Use YOLO class
#     model.to(device)
    
#     # Model optimization (optional, but good for inference speed)
#     # model.fuse() # Fuse conv-bn layers, standard optimization
#     # If using half precision
#     if device.startswith('cuda') and torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] >= 7:
#         print("Using half precision for model.")
#         model.half() # model.model.half() is for older ultralytics versions
#     else:
#         print("Not using half precision (either no CUDA, old GPU, or device check failed).")


#     results_list = []
#     for idx, tomo_id_str in enumerate(tqdm(tomos, desc="Processing Tomograms"), 1):
#         if device.startswith('cuda') and torch.cuda.is_available(): # Check CUDA availability again
#             torch.cuda.empty_cache()
        
#         # Pass total_tomos and current_tomo_idx if needed by process_tomogram
#         single_tomo_result = process_tomogram(tomo_id_str, model, len(tomos), idx)
#         results_list.append(single_tomo_result)

#     submission_df = pd.DataFrame(results_list)[['tomo_id','Motor axis 0','Motor axis 1','Motor axis 2']]
#     submission_df.to_csv(submission_path, index=False)
#     print("\nSubmission file created:")
#     print(submission_df.head())
#     return submission_df

# if __name__ == "__main__":
#     # Ensure CWD is correct if MHAF-YOLO has specific requirements
#     # (already handled at the top of the script by os.chdir(target_dir))
    
#     print(f"Running on device: {device}")
#     if device.startswith('cuda') and not torch.cuda.is_available():
#         print("Warning: Device set to CUDA but CUDA is not available. PyTorch will use CPU.")
#         device = 'cpu' # Fallback if check fails later

#     start_time = time.time()
#     generate_submission()
#     end_time = time.time()
#     print(f"\nTotal processing time: {end_time - start_time:.2f} seconds")